In [12]:
# pip install psycopg2-binary

     ---------------------------------------- 1.2/1.2 MB 10.6 MB/s eta 0:00:00
Note: you may need to restart the kernel to use updated packages.


In [40]:
import pandas as pd
import psycopg2 as pg
import sqlalchemy
from sqlalchemy.ext.automap import automap_base
from sqlalchemy.orm import Session
from sqlalchemy import create_engine, func
from sqlalchemy.orm import sessionmaker
from flask import Flask, jsonify, render_template
## generation personal config file as config.py
## variable has to be set to "pw"
from config import pw

In [41]:
connection_string = f"postgres:{pw}@localhost:5433/Austin_Housing_Market"
engine = create_engine(f'postgresql://{connection_string}')

In [42]:
Base = automap_base()

In [43]:
Base.prepare(engine, reflect=True)

annual_sales = Base.classes.annual_sales
monthly_sales = Base.classes.monthly_sales
price_distribution = Base.classes.price_distribution


In [44]:
session = Session(engine)


In [45]:
session.query(price_distribution.year_2011).__dict__

{'session': <sqlalchemy.orm.session.Session at 0x122cccc5d00>,
 '_propagate_attrs': immutabledict({'compile_state_plugin': 'orm', 'plugin_subject': <Mapper at 0x122cccfaa30; price_distribution>}),
 '_raw_columns': [Column('year_2011', DOUBLE_PRECISION(precision=53), table=<price_distribution>, nullable=False)]}

In [47]:
session.query(annual_sales).all()

In [54]:
# pull in annual home sales excel file and create dataframe
annual_data_df = pd.read_excel('Resources/annual_home_sales.xlsx')

annual_data_df.head()




,year,sales,dollar_volume,average_price,median_price,total_listings,months_inventory
0,1990,7068,579669083,82013,72252,4412,6.5
1,1991,7485,661674435,88400,75865,3552,4.6
2,1992,8389,832936284,99289,82929,3020,3.6
3,1993,9784,1076687995,110046,90949,2860,3.2
4,1994,10418,1207051547,115862,95158,3645,3.9


In [55]:
annual_data_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 33 entries, 0 to 32
Data columns (total 7 columns):
 #   Column            Non-Null Count  Dtype  
---  ------            --------------  -----  
 0   year              33 non-null     int64  
 1   sales             33 non-null     int64  
 2   dollar_volume     33 non-null     int64  
 3   average_price     33 non-null     int64  
 4   median_price      33 non-null     int64  
 5   total_listings    33 non-null     int64  
 6   months_inventory  33 non-null     float64
dtypes: float64(1), int64(6)
memory usage: 1.9 KB


In [50]:
# pull in monthly home sales excel file and create dataframe
monthly_data_df = pd.read_excel('Resources/monthly_home_sales.xlsx')
monthly_data_df.head()

,date,sales,dollar_volume,average_price,median_price,total_listings,months_inventory
0,1990-01-01,558,46937286,84117,71000,4391,7.9
1,1990-02-01,444,37056240,83460,71199,4343,8.7
2,1990-03-01,431,34722653,80563,72497,4459,9.3
3,1990-04-01,569,34616822,60838,71998,4545,9.1
4,1990-05-01,549,44907102,81798,73694,4731,9.3


In [52]:
# pull in price distribution excel file and create dataframe
price_data_df = pd.read_excel('Resources/price_distribution.xlsx')
price_data_df.head()


,price_distribution,2011,2012,2013,2014,2015,2016,2017,2018,2019,2020,2021,2022
0,"$0 - $69,999",4.5,2.9,1.7,0.9,0.6,0.4,0.2,0.1,0.1,0.1,0.0,0.0
1,"$70,000 - $99,999",6.5,5.3,3.3,2.0,1.1,0.8,0.5,0.3,0.2,0.2,0.0,0.0
2,"$100,000 - $149,999",21.6,19.5,16.0,11.0,6.5,3.7,2.5,1.7,1.2,0.8,0.3,0.1
3,"$150,000 - $199,999",21.0,21.4,22.3,22.2,19.5,15.2,11.2,7.9,6.5,3.8,1.3,0.4
4,"$200,000 - $249,999",13.0,14.0,15.2,17.0,18.6,19.6,20.6,20.6,19.9,14.7,3.8,1.1
